In [8]:
%matplotlib inline
import datetime
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import re
import time
from elasticsearch import Elasticsearch, helpers

In [9]:
last_days=150
es = Elasticsearch([{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

def time_filter(indices,pat=r'jobs_archive_', last_days=1):
    if last_days == 0:
        return ["jobs_archive_*"]
    today = datetime.date.today()
    filtered = []
    datefmt = '%Y-%m-%d'
    for i in indices:
        day = re.sub(pat, '', i).rstrip()
        day = datetime.datetime.strptime(day, datefmt).date()
        diff = today - day
        if diff.days < last_days:
            filtered.append(i.rstrip())
    return filtered

def get_indices(index="jobs_archive_*",pat=r'jobs_archive_',last_days=last_days):
    #indices_ja = es.cat.indices(index="jobs_archive_*", h="index", request_timeout=600).split('\n')
    indices = es.cat.indices(index=index, h="index", request_timeout=600).split('\n')
    indices = sorted(indices)
    indices = [x for x in indices if x != '' and not x.endswith('_reindexed')]
    #print("indices is",indices)
    if index=="boinc":
        ind=indices
    else:
        ind = time_filter(indices,pat=pat,last_days=last_days)
        print ("number of indices:", len(ind))
    ind = ','.join(ind)
    print(ind)
    return ind
ind_ja=get_indices()
ind_boinc=get_indices(index="boinc",pat=r'boinc_')

number of indices: 150
jobs_archive_2017-12-27,jobs_archive_2017-12-28,jobs_archive_2017-12-29,jobs_archive_2017-12-30,jobs_archive_2017-12-31,jobs_archive_2018-01-01,jobs_archive_2018-01-02,jobs_archive_2018-01-03,jobs_archive_2018-01-04,jobs_archive_2018-01-05,jobs_archive_2018-01-06,jobs_archive_2018-01-07,jobs_archive_2018-01-08,jobs_archive_2018-01-09,jobs_archive_2018-01-10,jobs_archive_2018-01-11,jobs_archive_2018-01-12,jobs_archive_2018-01-13,jobs_archive_2018-01-14,jobs_archive_2018-01-15,jobs_archive_2018-01-16,jobs_archive_2018-01-17,jobs_archive_2018-01-18,jobs_archive_2018-01-19,jobs_archive_2018-01-20,jobs_archive_2018-01-21,jobs_archive_2018-01-22,jobs_archive_2018-01-23,jobs_archive_2018-01-24,jobs_archive_2018-01-25,jobs_archive_2018-01-26,jobs_archive_2018-01-27,jobs_archive_2018-01-28,jobs_archive_2018-01-29,jobs_archive_2018-01-30,jobs_archive_2018-01-31,jobs_archive_2018-02-01,jobs_archive_2018-02-02,jobs_archive_2018-02-03,jobs_archive_2018-02-04,jobs_archive_2018

In [10]:
my_query_ja={
    "size": 0,
    "_source": ["modificationhost", "cpuconsumptiontime","nevents","jeditaskid","actualcorecount","wall_time","nevents"],
  "query": {
    "bool": {
      "must": [
        {
          "term": {"jobstatus": "finished"},
          "term": {"computingsite":"BOINC_MCORE"}
        }
      ]
    }
  }
}
my_query_boinc={
    "size": 0,
    "_source": ["result_app_version_id","batch","cpu_time","host_name","elapsed_time"],
  "query": {
    "bool": {
      "must": [
        {
          "term": {"outcome":1},
          "term": {"result_app_id":14}
        }
      ]
    }
  }
}


def list_res(res):
    count=0
    for r in res:
        if count<=10:
            print(r)
            count+=1


#get_avg_boinc_power_by_task(tasks,res)
def get_ja_info(res):
    ja_info={}
    for r in res:
        if "actualcorecount" in r["_source"] and r["_source"]["actualcorecount"]:
            taskid=r["_source"]["jeditaskid"]
            cputime=r["_source"]["cpuconsumptiontime"]
            walltime=r["_source"]["wall_time"]*r["_source"]["actualcorecount"]
            nevents=r["_source"]["nevents"]
            ncore=r["_source"]["actualcorecount"]
            host=r["_source"]["modificationhost"]
            if type(host)!=str or host=="":
                continue
            host=re.sub(".*@","",host)
            if cputime>2000*nevents or cputime<=0:
                #print("bad cputime",cputime)
                continue
            if walltime>2000*nevents*ncore or walltime<=0:
                #print("bad walltime",walltime)
                continue
            cpu_eff=np.round(cputime/walltime,2)
            if cpu_eff<1:
                #print(host,cputime,walltime,ncore,cpu_eff)
                if not host in ja_info:
                    ja_info[host]={}
                if not taskid in ja_info[host]:
                    ja_info[host][taskid]={}
                    ja_info[host][taskid]["cputime"]=0
                    ja_info[host][taskid]["walltime"]=0
                    ja_info[host][taskid]["cpu_eff"]=0
                    ja_info[host][taskid]["cn"]=0
                    ja_info[host][taskid]["ncore"]=0
                if ja_info[host][taskid]["ncore"]!=0 and ja_info[host][taskid]["ncore"]!=ncore:
                    continue
                ja_info[host][taskid]["cputime"]+=cputime
                ja_info[host][taskid]["walltime"]+=walltime
                ja_info[host][taskid]["cn"]+=1
                ja_info[host][taskid]["cpu_eff"]+=cpu_eff
                ja_info[host][taskid]["ncore"]=ncore
    #print(ja_info)
    return ja_info
def get_boinc_info(res):
    boinc_info={}
    for r in res:
        taskid=r["_source"]["batch"]
        host=r["_source"]["host_name"]
        cputime=r["_source"]["cpu_time"]
        walltime=r["_source"]["elapsed_time"]
        av=r["_source"]["result_app_version_id"]
        if walltime<=0:
            continue
        cpu_eff=np.round(cputime/walltime,2)
       
        if host=="":
            continue
        if not host in boinc_info:
            boinc_info[host]={}
        if not taskid in boinc_info[host]:
            boinc_info[host][taskid]={}
            boinc_info[host][taskid]["cputime"]=0
            boinc_info[host][taskid]["walltime"]=0
            boinc_info[host][taskid]["cn"]=0
            boinc_info[host][taskid]["av"]=0
            boinc_info[host][taskid]["cpu_eff"]=0
        if boinc_info[host][taskid]["av"]!=0 and av!=boinc_info[host][taskid]["av"]:
            continue
         
        boinc_info[host][taskid]["cputime"]+=cputime
        boinc_info[host][taskid]["walltime"]+=walltime
        boinc_info[host][taskid]["cn"]+=1
        boinc_info[host][taskid]["av"]=av
        boinc_info[host][taskid]["cpu_eff"]+=cpu_eff
    #print(boinc_info)
    return boinc_info

res_ja = helpers.scan(es, query=my_query_ja, index=ind_ja, scroll='5m', timeout="5m", size=1000)
ja_info=get_ja_info(res_ja)

res_boinc = helpers.scan(es, query=my_query_boinc, index=ind_boinc, scroll='5m', timeout="5m", size=1000)
 
boinc_info=get_boinc_info(res_boinc)    



In [11]:


dlist=[]
head=["av","ncore","taskid","host","panda_cn","boinc_cn","panda_cpusec_perjob","panda_cpu_eff","boinc_cpu_eff",\
      "cputime_diff","walltime_diff","job_cpu_eff"]
for host in ja_info.keys():
    for taskid in ja_info[host].keys():
        if host in boinc_info and taskid in boinc_info[host]:
            av=boinc_info[host][taskid]["av"]
            ncore=ja_info[host][taskid]["ncore"]
            boinc_cn=boinc_info[host][taskid]["cn"]
            ja_cn=ja_info[host][taskid]["cn"]
            if boinc_cn!=ja_cn:
                continue
            boinc_walltime=ncore*boinc_info[host][taskid]["walltime"]/boinc_cn
            panda_walltime=ja_info[host][taskid]["walltime"]/ja_cn
            boinc_cputime=boinc_info[host][taskid]["cputime"]/boinc_cn
            panda_cputime=ja_info[host][taskid]["cputime"]/ja_cn
            cpu_eff=ja_info[host][taskid]["cpu_eff"]/ja_cn
            cputime_perjob=ja_info[host][taskid]["cputime"]/ja_cn
            boinc_cpu_eff=boinc_info[host][taskid]["cpu_eff"]/boinc_cn
            boinc_cpu_eff=np.round(boinc_cpu_eff/ncore,2)
            job_cpu_eff=np.round(panda_cputime/boinc_walltime,2)
            if boinc_walltime<=0:
                continue
            if boinc_cpu_eff>cpu_eff:
                continue
            if boinc_walltime<panda_walltime-600:
                print("bad boinc walltime",boinc_walltime-panda_walltime)
                #boinc_walltime=panda_walltime
                continue
            if boinc_cputime<panda_cputime-600:
                print("bad boinc cputime",boinc_cputime-panda_cputime)
                #boinc_cputime=panda_cputime
                continue
            t=[av,ncore,taskid,host,boinc_cn,ja_cn,cputime_perjob,cpu_eff,boinc_cpu_eff,\
               boinc_cputime-panda_cputime,boinc_walltime-panda_walltime,job_cpu_eff]    
            dlist.append(t)
df=pd.DataFrame(dlist,columns=head)


df.to_csv("boinc_perf.csv")
df=df.sort_values(by="av",ascending=False)
print(df.describe())



bad boinc cputime -919.1999999999971
bad boinc cputime -34270.72
bad boinc cputime -9993.769375000003
bad boinc cputime -54729.259999999995
bad boinc cputime -5602.082926829258
bad boinc walltime -978.6870618750108
bad boinc cputime -4699.7345000000005
bad boinc walltime -1651.3664759999956
bad boinc walltime -2583.8385953333636
bad boinc cputime -1707.137499999997
bad boinc cputime -1546.979444444427
bad boinc cputime -2548.300000000003
bad boinc cputime -3386.266666666663
bad boinc cputime -3490.179999999993
bad boinc cputime -4794.581249999988
bad boinc cputime -3608.483333333337
bad boinc cputime -1474.1389999999956
bad boinc cputime -3746.9857142857218
bad boinc cputime -3820.850000000006
bad boinc cputime -1866.9134210526245
bad boinc cputime -2659.574999999997
bad boinc cputime -1777.1595833333267
bad boinc cputime -1773.6163414634066
bad boinc cputime -1458.234285714294
bad boinc cputime -906.9066666666658
bad boinc cputime -7012.643333333341
bad boinc cputime -7204.322
bad boi

bad boinc walltime -7136.968573714301
bad boinc walltime -3303.321015428577
bad boinc cputime -2486.1050909091136
bad boinc cputime -20942.445
bad boinc cputime -38989.95
bad boinc walltime -934.1583615999953
bad boinc walltime -2836.6159056000033
bad boinc cputime -6343.330000000002
bad boinc cputime -13792.029999999999
bad boinc cputime -2055.2233333333315
bad boinc cputime -8214.619999999995
bad boinc walltime -33113.442975
bad boinc cputime -617.3999999999978
bad boinc cputime -993.5400000000009
bad boinc cputime -1068.3099999999977
bad boinc cputime -608.8279999999977
bad boinc cputime -647.6599999999999
bad boinc cputime -1280.1759999999995
bad boinc walltime -1970.0294592941136
bad boinc cputime -1153.4550000000017
bad boinc walltime -2299.3362373333366
bad boinc cputime -1301.0299999999988
bad boinc cputime -651.5500000000029
bad boinc cputime -9732.32
bad boinc walltime -1180.0816539999942
bad boinc cputime -9164.36
bad boinc cputime -118459.5
bad boinc walltime -24106.7712799

bad boinc cputime -2538.5800000000017
bad boinc cputime -11954.529999999999
bad boinc walltime -6870.04078000001
bad boinc walltime -11450.862246400007
bad boinc walltime -68290.92728480001
bad boinc walltime -13925.985062260872
bad boinc walltime -18619.229746666664
bad boinc walltime -17931.18374800001
bad boinc cputime -1149.385000000002
bad boinc cputime -20057.82
bad boinc walltime -34972.92804
bad boinc cputime -1471.6000000000058
bad boinc cputime -2868.100000000006
bad boinc cputime -1161.4300000000003
bad boinc cputime -847.6150000000052
bad boinc cputime -53416.899999999994
bad boinc cputime -35844.7
bad boinc walltime -10431.432021400004
bad boinc walltime -1661.282296999998
bad boinc walltime -1219.0339789999998
bad boinc walltime -873.8812489999982
bad boinc cputime -24635.80249999999
bad boinc cputime -5811.18
bad boinc cputime -86175.93
bad boinc cputime -3711.4000000000233
bad boinc cputime -5510.0899999999965
bad boinc cputime -3631.904999999999
bad boinc cputime -3208

bad boinc cputime -638.0099999999984
bad boinc walltime -1109.8018653333274
bad boinc walltime -2637.017040000006
bad boinc walltime -3371.9217510000017
bad boinc walltime -11462.923994000012
bad boinc cputime -4458.080000000002
bad boinc cputime -4519.0899999999965
bad boinc cputime -11886.800000000003
bad boinc cputime -4935.199999999997
bad boinc cputime -3766.449999999997
bad boinc cputime -3982.779999999999
bad boinc walltime -5651.127884000001
bad boinc cputime -17269.65
bad boinc walltime -15072.630042000004
bad boinc walltime -4910.9288400000005
bad boinc walltime -14674.917793500004
bad boinc cputime -1123.6399999999994
bad boinc walltime -18419.896464999998
bad boinc cputime -26874.84
bad boinc walltime -123339.87776199999
bad boinc cputime -4992.690000000002
bad boinc cputime -5108.541250000002
bad boinc cputime -6165.309999999998
bad boinc cputime -6525.949999999997
bad boinc cputime -22624.899999999994
bad boinc cputime -5114.654999999999
bad boinc cputime -11346.766666666

bad boinc cputime -10304.05
bad boinc walltime -2747.632238000002
bad boinc walltime -7241.62604333333
bad boinc walltime -773.9102150000035
bad boinc walltime -10565.628731999997
bad boinc walltime -2599.595323499998
bad boinc walltime -20754.3125975
bad boinc walltime -1387.839097
bad boinc walltime -3693.925256600003
bad boinc cputime -8136.800000000003
bad boinc walltime -14694.842124000003
bad boinc cputime -5937.799999999999
bad boinc cputime -2123.875
bad boinc walltime -899.7312749999983
bad boinc cputime -13350.660000000003
bad boinc cputime -1548.6759999999995
bad boinc cputime -6014.6325
bad boinc walltime -3200.9703599999993
bad boinc cputime -7010.290000000001
bad boinc cputime -2249.4500000000007
bad boinc walltime -142364.060961
bad boinc walltime -30163.399439999994
bad boinc walltime -53864.852591999996
bad boinc cputime -927.7300000000014
bad boinc cputime -740.4400000000023
bad boinc cputime -36887.20000000001
bad boinc cputime -11733.11
bad boinc walltime -21813.591

bad boinc cputime -49223.48571428572
bad boinc cputime -100459.79999999996
bad boinc walltime -5791.998800000001
bad boinc cputime -67284.72727272726
bad boinc cputime -56986.479999999996
bad boinc cputime -124189.79999999999
bad boinc walltime -3358.3873279999243
bad boinc cputime -118663.93333333338
bad boinc cputime -73744.4
bad boinc cputime -81255.46666666665
bad boinc cputime -46109.139999999985
bad boinc cputime -59475.69999999998
bad boinc cputime -15082.119999999995
bad boinc cputime -4514.007142857146
bad boinc cputime -16211.100000000006
bad boinc cputime -12692.760000000002
bad boinc walltime -16020.136314666663
bad boinc cputime -1460.568181818182
bad boinc cputime -22190.559999999998
bad boinc cputime -719.0950000000012
bad boinc cputime -1748.0599999999977
bad boinc walltime -138354.2649
bad boinc walltime -128734.97876266667
bad boinc cputime -16320.550000000003
bad boinc cputime -41808.28
bad boinc cputime -2768.1699999999983
bad boinc cputime -58524.70000000001
bad bo

bad boinc cputime -1025.2239999999983
bad boinc cputime -1390.6549999999988
bad boinc cputime -667.2799999999988
bad boinc cputime -770.0
bad boinc cputime -1719.5950000000012
bad boinc cputime -15307.404999999999
bad boinc cputime -47258.41500000001
bad boinc cputime -82268.16
bad boinc cputime -64179.72
bad boinc cputime -19870.52833333335
bad boinc cputime -20942.9825
bad boinc walltime -1185.9962759999908
bad boinc cputime -30927.96666666669
bad boinc walltime -1044.537910153842
bad boinc cputime -7000.4100000000035
bad boinc walltime -1928.5987612499885
bad boinc cputime -2071.850000000006
bad boinc walltime -6778.382627000014
bad boinc walltime -9325.064416000001
bad boinc cputime -3250.6500000000015
bad boinc cputime -4588.470000000001
bad boinc cputime -1860.3349999999991
bad boinc cputime -3144.3050000000003
bad boinc cputime -4737.592500000006
bad boinc walltime -4780.317912000006
bad boinc walltime -2875.143039000017
bad boinc cputime -6065.32
bad boinc walltime -73478.83575

bad boinc walltime -28035.323200000013
bad boinc walltime -873.0773019999906
bad boinc walltime -604.6893986666691
bad boinc walltime -1188.7349836279027
bad boinc walltime -4735.227934117647
bad boinc cputime -2796.480909090911
bad boinc cputime -863.4100000000035
bad boinc cputime -660.7374999999993
bad boinc walltime -1504.1866659999941
bad boinc cputime -130012.92
bad boinc walltime -22867.731582000008
bad boinc cputime -1352.4700000000012
bad boinc cputime -3543.8099999999977
bad boinc cputime -5356.769999999997
bad boinc cputime -3503.25
bad boinc cputime -3892.5
bad boinc walltime -1493.5277559999959
bad boinc walltime -1635.0150719999947
bad boinc cputime -6505.5899999999965
bad boinc cputime -3817.9649999999965
bad boinc cputime -4265.449999999997
bad boinc cputime -47776.869999999995
bad boinc walltime -4819.461405000009
bad boinc cputime -35393.76999999999
bad boinc walltime -4456.612338000006
bad boinc cputime -35330.31
bad boinc cputime -34753.84
bad boinc walltime -6815.8

bad boinc walltime -52878.062013999996
bad boinc walltime -30814.299704999998
bad boinc walltime -125719.148214
bad boinc walltime -39113.290698
bad boinc walltime -4054.95846899999
bad boinc cputime -56296.81
bad boinc cputime -118959.7
bad boinc walltime -42972.853147999995
bad boinc cputime -100813.80000000002
bad boinc cputime -61414.229999999996
bad boinc cputime -43014.26
bad boinc cputime -43735.490000000005
bad boinc walltime -99868.26567599998
bad boinc cputime -969.5783333333311
bad boinc cputime -66655.76999999999
bad boinc cputime -120710.18
bad boinc cputime -1249.4400000000023
bad boinc walltime -9821.857825333325
bad boinc walltime -61655.22596
bad boinc cputime -2078.1849999999977
bad boinc cputime -900.0500000000029
bad boinc cputime -1090.75
bad boinc cputime -4673.799999999999
bad boinc cputime -1260.9400000000023
bad boinc cputime -2619.869999999999
bad boinc cputime -15068.350000000006
bad boinc cputime -5446.040000000001
bad boinc cputime -21262.510000000002
bad b